[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vcardonas/linkedin-project/blob/main/Cuadernos/3%20Traducci%C3%B3n_Datos.ipynb)

# **<center>PREPARACIÓN Y TRADUCCIÓN DE DATOS</center>**
**<center> ¿Qué habilidades y herramientas son las más demandadas para el puesto de Analista de Datos en Colombia? </center>**

## **LIBRERÍAS**

In [1]:
#Python == 3.10.8
import pandas as pd
import numpy as np

from langdetect import detect, detect_langs # Detectar idioma

from transformers import pipeline # Modelos

# Manejo de fechas
import dateparser
import datetime

## **IMPORTACIÓN DE DATOS SCRAPING**

In [2]:
# Importación
df = pd.read_csv('./Datos/ws_linkedin.csv')
df.head()

,Título,Empresa,Ubicación,Modalidad de trabajo,Publciación,Tipo de empleo,Descripción
0,Ruby on Rails Engineer,Aha!,"Bogotá, Distrito Capital, Colombia",En remoto,hace 12 minutos,Jornada completa · Sin experiencia,Acerca del empleo\nAha! is a very different ty...
1,DevOps Engineer - Remote - Colombia,FullStack Labs,"Cartagena de Indias, Bolívar, Colombia",En remoto,hace 5 días,Jornada completa · Sin experiencia,Acerca del empleo\nFullStack Labs is the faste...
2,"Product Developer, Retail Data Platform",Orium,Colombia,En remoto,hace 5 días,Jornada completa · Algo de responsabilidad,"Acerca del empleo\nSenior Product Developer, R..."
3,Trabajos remotos de Analista de Datos,Turing,Colombia,En remoto,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo\nUna empresa que cotiza en l...
4,Analista de datos y servicio al cliente,TH Plus Talento Humano,"Bogotá, Distrito Capital, Colombia",Presencial,hace 20 minutos,Jornada completa,"Acerca del empleo\nTH PLUS Talento Humano, req..."


In [3]:
# Reemplazar \n por espacios
df["Descripción"] = df["Descripción"].str.replace('\n'," ")
df["Descripción"]

0      Acerca del empleo Aha! is a very different typ...
1      Acerca del empleo FullStack Labs is the fastes...
2      Acerca del empleo Senior Product Developer, Re...
3      Acerca del empleo Una empresa que cotiza en la...
4      Acerca del empleo TH PLUS Talento Humano, requ...
                             ...                        
379    Acerca del empleo En NEORIS, acelerador digita...
380    Acerca del empleo FullStack Labs is the fastes...
381    Acerca del empleo DSM – Bright Science. Bright...
382    Acerca del empleo Job Description  Datavail he...
383    Acerca del empleo Construye y promueve excelen...
Name: Descripción, Length: 384, dtype: object

## **TRADUCCIÓN**

In [4]:
# Traductor
from transformers import pipeline
translator = pipeline("translation", model = "Helsinki-NLP/opus-mt-es-en")

In [5]:
## PREPARACIÓN DATOS PARA TRADUCCIÓN

# Columna vacía
df["descr_trans"] = None

# Duplicar en DF_EN
df["descr_trans"] = df["Descripción"].copy()

# Quitar "Acerca del empleo"
df["descr_trans"] = df["descr_trans"].str.split(n = 3).str[3]
df.head()

,Título,Empresa,Ubicación,Modalidad de trabajo,Publciación,Tipo de empleo,Descripción,descr_trans
0,Ruby on Rails Engineer,Aha!,"Bogotá, Distrito Capital, Colombia",En remoto,hace 12 minutos,Jornada completa · Sin experiencia,Acerca del empleo Aha! is a very different typ...,Aha! is a very different type of high-growth S...
1,DevOps Engineer - Remote - Colombia,FullStack Labs,"Cartagena de Indias, Bolívar, Colombia",En remoto,hace 5 días,Jornada completa · Sin experiencia,Acerca del empleo FullStack Labs is the fastes...,FullStack Labs is the fastest-growing software...
2,"Product Developer, Retail Data Platform",Orium,Colombia,En remoto,hace 5 días,Jornada completa · Algo de responsabilidad,"Acerca del empleo Senior Product Developer, Re...","Senior Product Developer, Retail Data Platform..."
3,Trabajos remotos de Analista de Datos,Turing,Colombia,En remoto,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo Una empresa que cotiza en la...,Una empresa que cotiza en la Bolsa de Nueva Yo...
4,Analista de datos y servicio al cliente,TH Plus Talento Humano,"Bogotá, Distrito Capital, Colombia",Presencial,hace 20 minutos,Jornada completa,"Acerca del empleo TH PLUS Talento Humano, requ...","TH PLUS Talento Humano, requiere para uno de s..."


In [6]:
# Detectar idioma
df['idioma'] = df["descr_trans"].apply(detect)

# Cantidad de descripciones por idioma
df['idioma'].value_counts().to_frame()

,idioma
es,196
en,181
pt,7


In [7]:
# Dividir DF por idioma
df_en = df[df['idioma'] == "en"]
df_es = df[df['idioma'] == "es"]

In [9]:
# Reestablecer índice
df_es.reset_index(inplace = True, drop = False)

# Traducir df_es
for i, row in enumerate(df_es.values):
    print(i, end = " ")
    try:
        df_es["descr_trans"].loc[i] = translator(row[8])
    except:
        pass
# No traducción de algunas descripciones
# -> "input_length: N is bigger than 0.9 * max_length: 512" (Su extensión es mayor a lo que el modelo puede traducir)

df_es

,index,Título,Empresa,Ubicación,Modalidad de trabajo,Publciación,Tipo de empleo,Descripción,descr_trans,idioma
0,3,Trabajos remotos de Analista de Datos,Turing,Colombia,En remoto,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo Una empresa que cotiza en la...,[{'translation_text': 'A company that is liste...,es
1,4,Analista de datos y servicio al cliente,TH Plus Talento Humano,"Bogotá, Distrito Capital, Colombia",Presencial,hace 20 minutos,Jornada completa,"Acerca del empleo TH PLUS Talento Humano, requ...","[{'translation_text': 'TH PLUS Human Talent, r...",es
2,7,Desarrollador de aplicaciones para móviles,Atianza,"Bogotá, Distrito Capital, Colombia",Presencial,hace 6 días,Jornada completa · Sin experiencia,Acerca del empleo 1. Cargo Requerido: - IOs Mo...,[{'translation_text': '1. Charge Required: - I...,es
3,8,Desarrollador de SW,SONDA,Área metropolitana de Medellín,Híbrido,hace 1 semana,Jornada completa · Algo de responsabilidad,Acerca del empleo Id de solicitud de puesto: ...,[{'translation_text': 'Job Application Id: 568...,es
4,9,Analista de Procesamiento de Datos,Kantar,"Bogotá, Distrito Capital, Colombia",Presencial,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo ¡Si eres tecnólogo o profesi...,[{'translation_text': 'If you are a technologi...,es
...,...,...,...,...,...,...,...,...,...,...
191,377,Analista de selección de personal,Inversiones Vaquita Express,"Perímetro Urbano Medellín, Antioquia, Colombia",Presencial,hace 1 semana,Jornada completa · Sin experiencia,Acerca del empleo Nos encontramos en la búsque...,[{'translation_text': 'We are looking for a pr...,es
192,378,Finance Intern,Eli Lilly and Company,"Providencia, Archipiélago de San Andrés, Provi...",En remoto,hace 5 días,Jornada completa · Sin experiencia,"Acerca del empleo At Lilly, we unite caring wi...","At Lilly, we unite caring with discovery to ma...",es
193,379,DATA SCIENCE SR CONS - HT,NEORIS,"Santiago, Putumayo, Colombia",Híbrido,hace 5 días,Jornada completa · Algo de responsabilidad,"Acerca del empleo En NEORIS, acelerador digita...","En NEORIS, acelerador digital que ayuda a las ...",es
194,381,Analista de Seguridad - Bogota,DSM,Colombia,Híbrido,hace 1 día,Jornada completa · Prácticas,Acerca del empleo DSM – Bright Science. Bright...,DSM – Bright Science. Brighter Living.™ Royal...,es


In [10]:
# Extraer los valores de los diccionarios (traducción) de cada fila
for i in range(len(df_es)):
    if type(df_es["descr_trans"].iloc[i]) == list:
        df_es["descr_trans"].iloc[i] = df_es["descr_trans"].iloc[i][0]["translation_text"]
    else:
        pass

# Restaurar índice
df_es.set_index("index", inplace = True)

# Eliminar nombre del índice
df_es.index.name = None

df_es.head()

/var/folders/89/g3h9cy012t9b461ggllqnbw00000gn/T/ipykernel_76125/2693799016.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_es["descr_trans"].iloc[i] = df_es["descr_trans"].iloc[i][0]["translation_text"]


,Título,Empresa,Ubicación,Modalidad de trabajo,Publciación,Tipo de empleo,Descripción,descr_trans,idioma
3,Trabajos remotos de Analista de Datos,Turing,Colombia,En remoto,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo Una empresa que cotiza en la...,A company that is listed on the New York Stock...,es
4,Analista de datos y servicio al cliente,TH Plus Talento Humano,"Bogotá, Distrito Capital, Colombia",Presencial,hace 20 minutos,Jornada completa,"Acerca del empleo TH PLUS Talento Humano, requ...","TH PLUS Human Talent, requires for one of its ...",es
7,Desarrollador de aplicaciones para móviles,Atianza,"Bogotá, Distrito Capital, Colombia",Presencial,hace 6 días,Jornada completa · Sin experiencia,Acerca del empleo 1. Cargo Requerido: - IOs Mo...,1. Charge Required: - IOs Mobile Engineer 2. F...,es
8,Desarrollador de SW,SONDA,Área metropolitana de Medellín,Híbrido,hace 1 semana,Jornada completa · Algo de responsabilidad,Acerca del empleo Id de solicitud de puesto: ...,Job Application Id: 56844 Vacancy Name: SW Dev...,es
9,Analista de Procesamiento de Datos,Kantar,"Bogotá, Distrito Capital, Colombia",Presencial,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo ¡Si eres tecnólogo o profesi...,If you are a technologist or a professional in...,es


In [11]:
# Unir DFs
frames = [df_es, df_en]
df = pd.concat(frames)

# Ordenando por índice
df = df.sort_index(ascending = True)

# Reestablecer índice
df.reset_index(inplace = True, drop = True)

# Renombrar columnas
df = df.rename(columns={'Publciación':'Publicación',
                        'descr_trans':'Descripción (inglés)'})

print(df.shape)
df.head()

(377, 9)


,Título,Empresa,Ubicación,Modalidad de trabajo,Publicación,Tipo de empleo,Descripción,Descripción (inglés),idioma
0,Ruby on Rails Engineer,Aha!,"Bogotá, Distrito Capital, Colombia",En remoto,hace 12 minutos,Jornada completa · Sin experiencia,Acerca del empleo Aha! is a very different typ...,Aha! is a very different type of high-growth S...,en
1,DevOps Engineer - Remote - Colombia,FullStack Labs,"Cartagena de Indias, Bolívar, Colombia",En remoto,hace 5 días,Jornada completa · Sin experiencia,Acerca del empleo FullStack Labs is the fastes...,FullStack Labs is the fastest-growing software...,en
2,"Product Developer, Retail Data Platform",Orium,Colombia,En remoto,hace 5 días,Jornada completa · Algo de responsabilidad,"Acerca del empleo Senior Product Developer, Re...","Senior Product Developer, Retail Data Platform...",en
3,Trabajos remotos de Analista de Datos,Turing,Colombia,En remoto,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo Una empresa que cotiza en la...,A company that is listed on the New York Stock...,es
4,Analista de datos y servicio al cliente,TH Plus Talento Humano,"Bogotá, Distrito Capital, Colombia",Presencial,hace 20 minutos,Jornada completa,"Acerca del empleo TH PLUS Talento Humano, requ...","TH PLUS Human Talent, requires for one of its ...",es


In [12]:
# Traducir Publicación
for i, row in enumerate(df.values):
    try:
        df["Publicación"].loc[i] = translator(row[4])
    except:
        pass

# Extraer translation_text
for i in range(len(df)):
    df["Publicación"].iloc[i] = df["Publicación"].iloc[i][0]["translation_text"]

df.head()

,Título,Empresa,Ubicación,Modalidad de trabajo,Publicación,Tipo de empleo,Descripción,Descripción (inglés),idioma
0,Ruby on Rails Engineer,Aha!,"Bogotá, Distrito Capital, Colombia",En remoto,12 minutes ago,Jornada completa · Sin experiencia,Acerca del empleo Aha! is a very different typ...,Aha! is a very different type of high-growth S...,en
1,DevOps Engineer - Remote - Colombia,FullStack Labs,"Cartagena de Indias, Bolívar, Colombia",En remoto,5 days ago,Jornada completa · Sin experiencia,Acerca del empleo FullStack Labs is the fastes...,FullStack Labs is the fastest-growing software...,en
2,"Product Developer, Retail Data Platform",Orium,Colombia,En remoto,5 days ago,Jornada completa · Algo de responsabilidad,"Acerca del empleo Senior Product Developer, Re...","Senior Product Developer, Retail Data Platform...",en
3,Trabajos remotos de Analista de Datos,Turing,Colombia,En remoto,4 weeks ago,Jornada completa · Sin experiencia,Acerca del empleo Una empresa que cotiza en la...,A company that is listed on the New York Stock...,es
4,Analista de datos y servicio al cliente,TH Plus Talento Humano,"Bogotá, Distrito Capital, Colombia",Presencial,20 minutes ago,Jornada completa,"Acerca del empleo TH PLUS Talento Humano, requ...","TH PLUS Human Talent, requires for one of its ...",es


## **PRE-PROCESAMIENTO DE DATOS**

In [13]:
# Copia para poder trabajar con ella
df2 = df.copy()

In [14]:
# Convertir "hace x días/semanas/meses/etc" a fecha
for i in range(len(df2)):
    fecha = df2["Publicación"].iloc[i]
    fecha_ws = datetime.datetime.strptime('28/11/22 16:43','%d/%m/%y %H:%M')
    fecha = dateparser.parse(fecha, settings = {'RELATIVE_BASE': fecha_ws})
    fecha = fecha.strftime('%d/%m/%y')
    df2["Publicación"].iloc[i] = fecha

# Ordenar por columna de Publiación
df2 = df2.sort_values(by = "Publicación")

# Eliminar duplicados y conservar la última fecha
print(df2.shape) # Dimensiones antes
df2 = df2.drop_duplicates(subset = ["Descripción", "Empresa"], keep = 'last')
print(df2.shape) # Dimensiones después

# Reestablecer índice
df2.reset_index(inplace = True, drop = True)

df2.head()

/Users/valentinacardona/opt/anaconda3/envs/Proyecto_Diplomado/lib/python3.10/site-packages/dateparser/freshness_date_parser.py:76: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  now = self.get_local_tz().localize(now)


(377, 9)
(344, 9)


,Título,Empresa,Ubicación,Modalidad de trabajo,Publicación,Tipo de empleo,Descripción,Descripción (inglés),idioma
0,Ingeniero Junior de Automatización,Kent,"Bogotá, Distrito Capital, Colombia",Híbrido,07/11/22,Jornada completa · Algo de responsabilidad,Acerca del empleo Acerca de usted: Si tiene a...,Acerca de usted: Si tiene ambiciones de clase...,es
1,Work From Home Golang Developer,Workling,"Cali, Valle del Cauca, Colombia",En remoto,07/11/22,Jornada completa · Algo de responsabilidad,Acerca del empleo Location: Remote WFH All Lat...,Location: Remote WFH All Latin American Candid...,en
2,Developer Support Engineer Student Worker,Unity,"Perimetro Urbano Pereira, Risaralda, Colombia",Presencial,07/11/22,Jornada completa · Sin experiencia,Acerca del empleo The opportunity Unity’s Eng...,The opportunity Unity’s Engine Support Team i...,en
3,Practicante Profesional,Banco AV Villas,"Bogotá, Distrito Capital, Colombia",Híbrido,07/11/22,Jornada completa · Algo de responsabilidad,Acerca del empleo El Banco Comercial AV Villas...,The Commercial Bank AV Villas believe that edu...,es
4,Influencer Marketing Specialist,Edelman,"Bogotá, Distrito Capital, Colombia",Presencial,07/11/22,Jornada completa · Algo de responsabilidad,Acerca del empleo En Edelman estamos buscando ...,En Edelman estamos buscando un Influencer Mark...,es


In [15]:
# Dividir Columna TIPO DE EMPLEO
df2[['Jornada laboral', 'Nivel de experiencia']] = df2['Tipo de empleo'].str.split(' · ', 1, expand = True)

# Corregir valores que de Nivel de experiencia que terminaron en Jornada laboral
print(df2["Nivel de experiencia"].unique())
print(df2["Jornada laboral"].unique())

cat_exp = df2["Nivel de experiencia"].unique().tolist()

for i in range(len(df2)):
    if (df2["Nivel de experiencia"].iloc[i] is None) & (df2["Jornada laboral"].iloc[i] in cat_exp):
        df2["Nivel de experiencia"].iloc[i] = df2["Jornada laboral"].iloc[i]
        df2["Jornada laboral"].iloc[i] = None

print(df2["Jornada laboral"].unique())

['Algo de responsabilidad' 'Sin experiencia' 'Prácticas' None]
['Jornada completa' 'Media jornada' 'Algo de responsabilidad' 'Prácticas']
['Jornada completa' 'Media jornada' None 'Prácticas']


/var/folders/89/g3h9cy012t9b461ggllqnbw00000gn/T/ipykernel_76125/4122820458.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df2[['Jornada laboral', 'Nivel de experiencia']] = df2['Tipo de empleo'].str.split(' · ', 1, expand = True)


In [16]:
# Dividir Columna UBICACIÓN
df2['Municipio'] = None
df2['Departamento'] = None
df2['País'] = None

for i in range(len(df2)):
    if len(df2["Ubicación"].iloc[i].split(', ')) == 3:
        df2.loc[i][['Municipio', 'Departamento', 'País']] = df2['Ubicación'].iloc[i].split(', ', 2)
    elif len(df2["Ubicación"].iloc[i].split(', ')) == 2:
        df2.loc[i][['Departamento', 'País']] = df2['Ubicación'].iloc[i].split(', ', 1)
    else:
        df2['País'].iloc[i] = df2['Ubicación'].iloc[i]

In [17]:
# Eliminar columnas
df2.drop(["Tipo de empleo", "Ubicación"], axis = 1, inplace = True)

# Detectar idioma
df2['idioma'] = df2["Descripción (inglés)"].apply(detect)

# Cantidad de descripciones por idioma
df2['idioma'].value_counts().to_frame()

,idioma
en,290
es,54


In [18]:
# Excluir idioma español
df2 = df2[df2['idioma'] == "en"]

# Verificación cantidad de descripciones por idioma
df2['idioma'].value_counts().to_frame()

,idioma
en,290


In [19]:
# Información Variables
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290 entries, 1 to 343
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Título                290 non-null    object
 1   Empresa               290 non-null    object
 2   Modalidad de trabajo  290 non-null    object
 3   Publicación           290 non-null    object
 4   Descripción           290 non-null    object
 5   Descripción (inglés)  290 non-null    object
 6   idioma                290 non-null    object
 7   Jornada laboral       289 non-null    object
 8   Nivel de experiencia  285 non-null    object
 9   Municipio             201 non-null    object
 10  Departamento          214 non-null    object
 11  País                  290 non-null    object
dtypes: object(12)
memory usage: 29.5+ KB


## **EXPORTACIÓN DE DATOS**

In [20]:
# Tabla de datos final
df2.head()

,Título,Empresa,Modalidad de trabajo,Publicación,Descripción,Descripción (inglés),idioma,Jornada laboral,Nivel de experiencia,Municipio,Departamento,País
1,Work From Home Golang Developer,Workling,En remoto,07/11/22,Acerca del empleo Location: Remote WFH All Lat...,Location: Remote WFH All Latin American Candid...,en,Jornada completa,Algo de responsabilidad,Cali,Valle del Cauca,Colombia
2,Developer Support Engineer Student Worker,Unity,Presencial,07/11/22,Acerca del empleo The opportunity Unity’s Eng...,The opportunity Unity’s Engine Support Team i...,en,Jornada completa,Sin experiencia,Perimetro Urbano Pereira,Risaralda,Colombia
3,Practicante Profesional,Banco AV Villas,Híbrido,07/11/22,Acerca del empleo El Banco Comercial AV Villas...,The Commercial Bank AV Villas believe that edu...,en,Jornada completa,Algo de responsabilidad,Bogotá,Distrito Capital,Colombia
4,Influencer Marketing Specialist,Edelman,Presencial,07/11/22,Acerca del empleo En Edelman estamos buscando ...,En Edelman estamos buscando un Influencer Mark...,en,Jornada completa,Algo de responsabilidad,Bogotá,Distrito Capital,Colombia
5,Data Engineer,Kin + Carta,Presencial,07/11/22,"Acerca del empleo At Kin + Carta, we’ve got op...","At Kin + Carta, we’ve got opportunities to off...",en,Jornada completa,Algo de responsabilidad,Colombia,Huila,Colombia


In [21]:
# Almacenar los datos en un df
df2.to_csv('trans_linkedin.csv', index = False)